In [10]:
import pandas as pd
import numpy as np
import copy, time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

df_train = pd.read_csv('library_train.csv',encoding='utf-8')
df_test = pd.read_csv('library_test.csv',encoding='utf-8')

train_Y = df_train['排名差']
ids = df_test['學生']
df_train = df_train.drop(['學生', '排名差'] , axis=1)
df_test = df_test.drop(['學生'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,學年,年級,學期,概化排名
0,100,1,1,5
1,100,1,2,5
2,101,2,1,5
3,101,2,2,5
4,102,3,1,5


In [11]:
df = pd.get_dummies(df, columns = ["學年"])
df = pd.get_dummies(df, columns = ["年級"])
df = pd.get_dummies(df, columns = ["學期"])
df = pd.get_dummies(df, columns = ["概化排名"])

In [12]:
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

In [4]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True)
rf.fit(train_X, train_Y)
rf_pred = rf.predict(test_X)

In [13]:
xgb = xgb.XGBRegressor(base_score=0.5,colsample_bylevel=1,colsample_bytree=1,gamma=0,learning_rate=0.1,max_delta_step=0,max_depth=3,min_child_weight=1,
                      missing=None,n_estimators=100,nthread=-1,objective='reg:linear',reg_alpha=0,reg_lamba=1,scale_pos_weight=1,seed=1850,silent=True,subsample=1)
xgb.fit(train_X, train_Y)
xgb_pred = xgb.predict(test_X)

[18:02:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:02:34] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { reg_lamba, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:02:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [14]:
df_result = pd.read_csv('submit.csv',encoding='utf-8')
df_result.shape

(2179, 2)

In [15]:
df_result['排名差'] = xgb_pred

In [16]:
df_result.to_csv('submit3_xgb.csv',index = False)

In [17]:
print(df_result.head(10))

                                         學生       排名差
0  003322731d5df9d33d40b43ec5c300f89d956c44  2.053652
1  007a91bb90a8bfb3ac4ba999d252a35b824cbb8e  0.240195
2  0089869d4d8a20cc6b5ff2627152b9da60343469  3.497874
3  00c6deac1fdfde67fb46f73e1fc29ffb48526762  3.497874
4  00d718e3d99e68d643d3f76048a567a48249395b  3.338294
5  00d98d181668cf1e6b19378f9d540a2587ea3726  3.338294
6  010292328eb39d9d8842cb15c93d6185652460e5  2.053652
7  010338946952be09a66cc3cba1d162724b2e57ee  3.497874
8  0108689691f37c59a498c52f1f95f59cbcc6e723  0.240195
9  0127cac9bc4da58373fb14fbfdc9e77207c19e0d -3.438568
